# Exam 2487 B coding (40 points)

This part of the exam will carry 40 points, while the theoretical part will carry 60 points. You have in total 3 hours to complete both parts. For the theoretical part, you will complete the moodle quiz, while for part you will submit this notebook through moodle assignment. 

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# import sklearn packages for modelling
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, fbeta_score
from sklearn.metrics import recall_score,precision_score,f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.utils import resample
from sklearn.metrics import roc_curve
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
import missingno as msno
from sklearn.inspection import permutation_importance
from sklearn import metrics
import category_encoders as ce
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from scipy.stats import uniform as sp_randFloat
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Import ML-Classifiers
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier

# Impmort ML-Regressors
from catboost import CatBoostRegressor 
from lightgbm import LGBMRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

# Import NN
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

# Data Visualization set up
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import Image, display
large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")
%matplotlib inline
%config InlineBackend.figure_format='retina'

#setting max number of columns to be displayed to 100, to get a better view.
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

# Model Auditing
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
from aequitas.plotting import Plot
%matplotlib inline
%load_ext autoreload
%autoreload

## Question 1 (26 points)

You need to predict whether an individual’s annual income exceeds 50,000USD using individuals' social and demographic attributes. Annual income is the target feature and is measured as a binary category target; either less than or equal to 50kUSD, or greater than 50kUSD. The target is found in the column called `income`, positive class should be having income greater than 50kUSD, and negative otherwise.

Please import the dataset  `question1_data.csv` and perform the following steps: 

a) Check the number of data instances, number of variables and data types of variables. Convert the income greater than 50kUSD  to target value 1, and 0 otherwise.  **(2pts)** 

b) Divide the dataset into 50% training set and 50% test set.  **(1pt)** 

c) Check if any of the variables have missing data, and if so, for categorical variables impute the missing values with the most frequent category, and for numerical data, impute the missing values with the mean. **(3pts)** 

d) Encode the categorical variables using one hot encoding (you can decide whether or not to drop one dummy column, based on the following steps). **(2pts)** 

e) Scale the data to be in the range between 0 and 1. **(2pts)** 

f) Pick a classifier, non ensemble, and tune it using 5-fold cross validation. The scoring metric should be-F1 score. You should tune two different parameters, 2 values each (in total 4 parameter combinations). **(4pts)** 

g) Pick an ensemble classifier and tune it using 5-fold cross validation. The scoring metric should be F1-score. You should tune two different parameters, 2 values each (in total 4 parameter combinations). **(4pts)** 

h) Pick one best model according to the F1 metric from the models tuned in the previous two steps. Evaluate its performance on the test set, by printing out the classification report and plotting the ROC curve. **(2pts)** 

i) Perform Principal Component Analysis (PCA) on the train data, processed as before (imputation of missing values, one hot encoding of categorical variables, scaling) and select the number of components that capture at least 95% of variance.  **(3pts)** 

j) Retrain the model using the selected number of the PCA components on the train data. You do not need to tune the model, only to use the same model and parameters as selected in the previous steps. Evaluate the new model again on the test data: print the classification report and plot the ROC curve. **(3pts)** 


Important notes:
- You can change the order of steps or combine them in order as you see appropriate to avoid data leakage, which will be penalized.
- Each step is graded seperately, and points will be given for correct steps even if the previous steps were not done correctly. For example, if you delete the missing values, but do the remainder of the tasks correctly, you will loose only points awarded for imputation.
- You do not need to balance the data, or check for outliers. 



In [2]:
df_Q1 = pd.read_csv('question1_data.csv')
df_Q1['TARGET'] = df_Q1['income'].apply(lambda x: 0 if x == "<=50K" else 1)
df_Q1.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,income,TARGET
0,39.0,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,<=50K,0
1,50.0,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,<=50K,0
2,38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,<=50K,0
3,53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,<=50K,0
4,28.0,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,<=50K,0


#### b)

In [3]:
X = df_Q1.drop(['income', 'TARGET'], axis=1)
y = df_Q1['TARGET']


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1993)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(16280, 13)
(16281, 13)
(16280,)
(16281,)


In [5]:
def missing_values_table(df):
    # Total missing values
    mis_val = df.isnull().sum()

    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
        columns={0: 'Missing Values', 1: '% of Total Values'})

    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = (mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:, 1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1))

    # Print some summary information
    print("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
          "There are " + str(mis_val_table_ren_columns.shape[0]) +
          " columns that have missing values.")

    # Return the dataframe with missing information
    return mis_val_table_ren_columns

table_missing_features = missing_values_table(df_Q1)
table_missing_features.head(13)
missing_values = missing_values_table(df_Q1).reset_index()
missing_values_cols = missing_values[missing_values['Missing Values'] > 50]
missing_values_cols = missing_values_cols['index'].to_list()

Your selected dataframe has 15 columns.
There are 13 columns that have missing values.


,Missing Values,% of Total Values
workclass,351,1.1
capital-gain,351,1.1
hours-per-week,340,1.0
race,335,1.0
occupation,330,1.0
marital-status,328,1.0
gender,326,1.0
education-num,319,1.0
education,318,1.0
relationship,309,0.9


Your selected dataframe has 15 columns.
There are 13 columns that have missing values.


In [6]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot',  OneHotEncoder())])

numeric_features = X.select_dtypes(include=np.number).columns
categorical_features = X.select_dtypes(exclude=['number']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

f) Pick a classifier, non ensemble, and tune it using 5-fold cross validation. The scoring metric should be-F1 score. You should tune two different parameters, 2 values each (in total 4 parameter combinations). **(4pts)** 

In [7]:
tree = DecisionTreeClassifier(random_state = 1993)
clf_tree = Pipeline([('preprocessor', preprocessor), ('tree', tree)])  

tree_params = {"tree__max_depth": [3, 4],
               "tree__criterion": ["gini", "entropy"]}

clf_tree = RandomizedSearchCV(estimator=clf_tree,
                              param_distributions = tree_params,
                              cv=5, 
                              scoring= 'f1', 
                              n_jobs = -1, 
                              return_train_score=True, 
                              random_state=2021,
                              refit= True)

In [8]:
clf_tree.fit(X_train, y_train);

print('Final score on training-set is: ', clf_tree.score(X_train, y_train))
print('Final score on test-set is: ', clf_tree.score(X_test, y_test))
print (' ')
print('\nBest params:\n', clf_tree.best_params_)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer()),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')),
                                                                              ('cat',
                                                                               Pipeline(steps=[

Final score on training-set is:  0.616910866910867
Final score on test-set is:  0.5996629385628927
 

Best params:
 {'tree__max_depth': 4, 'tree__criterion': 'gini'}


g) Pick an ensemble classifier and tune it using 5-fold cross validation. The scoring metric should be F1-score. You should tune two different parameters, 2 values each (in total 4 parameter combinations). **(4pts)** 

In [13]:
rforest = RandomForestClassifier(random_state=2021)
clf_rforest = Pipeline([('preprocessor', preprocessor), ('rforest', rforest)])

param_grid = {
    'rforest__max_depth': [80, 100],
    'rforest__n_estimators': [100, 200]
            }

clf_rforest = RandomizedSearchCV(
                            estimator = clf_rforest, 
                            param_distributions = param_grid, 
                            cv=5, 
                            n_jobs = -1, 
                            scoring= 'f1',
                            return_train_score=True, 
                            random_state=1993)


clf_rforest.fit(X_train, y_train);

print('Final score on training-set is: ', clf_rforest.score(X_train, y_train))
print('Final score on test-set is: ', clf_rforest.score(X_test, y_test))
print (' ')
print('\nBest params:\n', clf_rforest.best_params_)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer()),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')),
                                                                              ('cat',
                                                                               Pipeline(steps=[

Final score on training-set is:  0.9998725302740599
Final score on test-set is:  0.668326417704011
 

Best params:
 {'rforest__n_estimators': 100, 'rforest__max_depth': 80}


h) Pick one best model according to the F1 metric from the models tuned in the previous two steps. Evaluate its performance on the test set, by printing out the classification report and plotting the ROC curve. **(2pts)** 

The random forest performs better on the training set. With an f1 score of 0.999 it clearly beats the descion tree. The choosen model based on the results of the training set and CV = 5. 

The papramteres are:
estimations = 100; max depth = 80

i) Perform Principal Component Analysis (PCA) on the train data, processed as before (imputation of missing values, one hot encoding of categorical variables, scaling) and select the number of components that capture at least 95% of variance.  **(3pts)** 

In [15]:
from sklearn.decomposition import PCA
# keep the first two principal components of the data
pca = PCA()
# fit PCA model to breast cancer data
pca.fit()

ValueError: could not convert string to float: ' Private'

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In [ ]:
np.cumsum(pca.explained_variance_ratio_)

## Question 2 (14 points)
You need to perform clustering using the dataset `question2_data.csv`. Please perform the following steps: 

a) import the dataset and and check the number of data instances, number of variables and data types of variables **(2pts)** 

b) Scale the data to achieve zero mean and unit variance. **(2pts)** 

b) Perform Kmeans for number of clusters varying from 2 to 10, inclusively. **(2pts)** 

c) Using two different metrics of your choice, select an appropriate number of clusters, justify it, and rerun the clustering for the selected number of clusters. **(3pts)** 

d) Visualize the clusters from the previous step using TSNE method, where each cluster should be plotted with a different color. **(2pts)** 

e) Perform DBSCAN clustering on the scaled dataset with 4 different parameter values and report the number of clusters found for each parameter value. **(3pts)**


In [ ]:
df_Q2 = pd.read_csv("question2_data.csv")
df_Q2.head()

In [ ]:
df_Q2.info()

In [ ]:
df_Q2.shape

#### b)

In [ ]:
scaler = StandardScaler()
df_Q2_scaled = scaler.fit_transform(df_Q2)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans_kwargs = {
      "init": "random",
       "n_init": 10,
       "max_iter": 300,
       "random_state": 1993,
 }

sse = []
for k in range(1, 11):
     kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
     kmeans.fit(df_Q2_scaled)
     sse.append(kmeans.inertia_)

In [ ]:
plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

In [ ]:
from sklearn.metrics import davies_bouldin_score

DB_score = []

for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(df_Q2_scaled)
    score = davies_bouldin_score(df_Q2_scaled, kmeans.labels_)
    DB_score.append(score)

In [ ]:
plt.style.use("fivethirtyeight")
plt.plot(range(2, 11), DB_score)
plt.xticks(range(2, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("Davies Bouldin Score")
plt.show()

From the SSE curve, 2 or 5 cluster seem to be the most reasonable. Based on the Elbow rule. The gardient switch here the most.

From the Davies Bouldin Score 5 cluster seems the best choice. This choice is made due to the minimum value of the score at 5 clusters.

Combined 5 cluster seem to be a legit choice.

This is a good example why to consult diffrent metrics is important.

In [ ]:
plt.figure(1)
plt.scatter(df_Q2_scaled[:, 0],df_Q2_scaled[:, 1], s=30) 
plt.title("our dataset")
plt.show()

In [ ]:
kmeans = KMeans(
           init="random",
           n_clusters=5,
           n_init=10,
           max_iter=300,
           random_state=1993
 )


In [ ]:
kmeans.fit(df_Q2_scaled)

In [ ]:
plt.figure(1)
plt.scatter(df_Q2_scaled[:, 0], df_Q2_scaled[:, 1], s=30, c=kmeans.labels_) 
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=60, alpha=0.5, marker='*')
plt.title("clusters found by kmeans")
plt.show()

#### DBScan

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
dbscan = DBSCAN(eps=0.1)
dbscan.fit(df_Q2_scaled)

In [ ]:
plt.figure(1)
plt.scatter(df_Q2_scaled[:, 0], df_Q2_scaled[:, 1], s=30, c=dbscan.labels_) 
plt.title("clusters found by dbscan")
plt.show()

In [ ]:
n_clusters_ = len(set(dbscan.labels_)) - (1 if -1 in dbscan.labels_ else 0)
n_clusters_

In [ ]:
para = [0.1, 0.3, 0.5, 10]
for i in para:
    dbscan = DBSCAN(eps=0.1)
    dbscan.fit(df_Q2_scaled)
    n_clusters_ = len(set(dbscan.labels_)) - (1 if -1 in dbscan.labels_ else 0)
    print("eps: ", i,"Cluster: ", n_clusters_)